In [ ]:
#--------------------------------------------------------------------------------
# SignalFlow: Python Percussion Workshop examples
#--------------------------------------------------------------------------------

In [ ]:
from signalflow import *

graph = AudioGraph()

In [ ]:
#--------------------------------------------------------------------------------
# HI-HAT
# Simple example using enveloped noise and a resonant high-pass filter.
#--------------------------------------------------------------------------------

In [ ]:
class HiHatPatch (Patch):
    def __init__(self, amp=0.3, hpf=10000, decay=0.01, resonance=0.7):
        super().__init__()
        noise = WhiteNoise()
        noise = SVFilter(noise, "high_pass", hpf, resonance)
        env = ASREnvelope(0.001, 0.01, decay)
        output = StereoPanner(noise * (env ** 3) * amp)
        self.set_output(output)
        self.set_auto_free_node(env)

In [ ]:
hat = HiHatPatch(amp=0.5, hpf=10000, resonance=0.9, decay=0.3)
hat.play()

In [ ]:
#--------------------------------------------------------------------------------
# KICK DRUM
# Models the body of the drum with low-frequency sine wave with decreasing 
# frequency, and the strike with a short burst of low-pass filtered noise.
#--------------------------------------------------------------------------------

In [ ]:
class KickDrumPatch(Patch):
    def __init__(self):
        super().__init__()
        duration = 0.2

        #--------------------------------------------------------------------------------
        # Transient (noise): Models the initial strike
        #--------------------------------------------------------------------------------
        transient = WhiteNoise() * ASREnvelope(0, 0, 0.03)
        transient = SVFilter(transient, "low_pass", 800)

        #--------------------------------------------------------------------------------
        # Body (tone): Models the tonal resonance of the body
        #--------------------------------------------------------------------------------
        frequency = Line(70, 40, duration)
        sine = SineOscillator(frequency)
        envelope = ASREnvelope(0.0, 0.0, duration)
        body = sine * envelope

        kick = (0.5 * transient) + body
        output = StereoPanner(kick)
        self.set_output(output)
        self.set_auto_free_node(envelope)
kick = KickDrumPatch()
kick.play()

In [ ]:
kick = KickDrumPatch()
kick.play()

In [ ]:
#----------------------------------------------------------------------------------------------------
# TOM
#
# Passes a short pulse through a resonant band-pass filter to create a pitched tom drum.
# Resonance controls the duration of the tail.
#
# The midi_note_to_frequency helper function is used to map the output to a MIDI pitch.
# Because we don't know when the note is going to end, DetectSilence is used to automatically
# free the node when the tail has finished sounding.
#----------------------------------------------------------------------------------------------------

In [ ]:
class TomPatch (Patch):
    def __init__(self, amp=0.3, resonance=0.99, note=56):
        super().__init__()
        impulse = ASREnvelope(0, 0, 0.01)
        resonant = SVFilter(impulse, "band_pass", midi_note_to_frequency(note), resonance=resonance)
        output = StereoPanner(resonant * amp)
        detect_silence = DetectSilence(output, threshold=0.00001)
        self.set_output(detect_silence)
        self.auto_free_node = detect_silence

In [ ]:
tom = TomPatch(resonance=0.99, note=56)
tom.play()

In [ ]:
#----------------------------------------------------------------------------------------------------
# HI-HAT V2
# 
# A more sophisticated hi-hat algorithm, taken from the approach used by some early Roland
# drum machines:
# 
# More information:
# https://www.soundonsound.com/techniques/practical-cymbal-synthesis
# https://www.cim.mcgill.ca/~clark/nordmodularbook/nm_percussion.html
#----------------------------------------------------------------------------------------------------

In [ ]:
class HiHatPatch2 (Patch):
    def __init__(self, decay=0.2, hpf=6000, resonance=0.7):
        super().__init__()
        freqs = [1, 1.1414, 1.1962, 2.1430, 2.4961, 2.0558]
        squares = [SquareOscillator(freq * 100) for freq in freqs]
        square = Sum(squares)
        filter1 = SVFilter(square, "band_pass", hpf, resonance)
        filter2 = SVFilter(filter1, "band_pass", hpf + 2840, resonance)
        envelope = ASREnvelope(0.01, 0, decay + 0.1, curve=7)
        output = filter2 * envelope * 0.1
        output = StereoPanner(output) 
        self.set_output(output)
        self.set_auto_free_node(envelope)

In [ ]:
hat2 = HiHatPatch2(decay=1.0, hpf=6000, resonance=0.9)
hat2.play()

In [ ]:
#----------------------------------------------------------------------------------------------------
# CLAP
# Generate a short noise pulse, and play multiple repeats of the pulse with random delays.
#----------------------------------------------------------------------------------------------------

In [ ]:
class ClapPatch (Patch):
    def __init__(self):
        super().__init__()
        claps = []
        for n in range(4):
            noise = WhiteNoise()
            delay = 0.01 if n < 3 else 0.05
            envelope = ASREnvelope(0.0, 0.02, delay)
            envelope_shaped = envelope ** 4
            output = noise * envelope_shaped
            delayed = OneTapDelay(output, 0.03 * n + RandomUniform(0.0, 0.01, clock=0))
            claps.append(delayed)
        
        total = Sum(claps) * 0.5
        total = StereoPanner(total)
        total = SVFilter(total, "high_pass", 5000, 0.3)
        total = SVFilter(total, "low_pass", 9000, 0.5)
        long_env = ASREnvelope(0, 0, 0.5)
        self.set_output(total * long_env)
        self.set_auto_free_node(long_env)

In [ ]:
clap = ClapPatch()
clap.play()

In [ ]:
#----------------------------------------------------------------------------------------------------
# SEQUENCING
# You will need to install the isobar sequencing library for this.
# Sequencing is currently laggy and problematic on Windows - apologies!
#----------------------------------------------------------------------------------------------------
%pip install isobar

In [ ]:
import isobar

In [ ]:
#----------------------------------------------------------------------------------------------------
# Create a timeline at 120bpm to sequence events
#----------------------------------------------------------------------------------------------------
output_device = isobar.SignalFlowOutputDevice(graph)
timeline = isobar.Timeline(120, output_device=output_device)
timeline.background()

In [ ]:
kick_track = timeline.schedule({
    "patch": KickDrumPatch,
    "duration": 1.0
}, quantize=1.0, name="kick", replace=True)

hat_track = timeline.schedule({
    "patch": HiHatPatch2,
    "duration": 0.25,
    "params": {
        "decay": isobar.PSequence([0, 0, 2, 0, 0.5, 0, 0.2, 0]) * 0.2 + 0.01,
        "hpf": isobar.PScaleLinExp(isobar.PBrown(0.5, 0.05, 0, 1), 0, 1, 6000, 15000),
    }
}, quantize=1.0, name="hat", replace=True)

tom_track = timeline.schedule({
    "patch": TomPatch,
    "duration": 0.25,
    "params": {
        "note": isobar.PSequence([40, 40, 48, 48, 43]) + 12,
        "resonance": isobar.PWhite(0.98, 0.99),
    },
    "active": isobar.PSequence([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]),
}, quantize=1.0, name="tom", replace=True)

clap_track = timeline.schedule({
    "patch": ClapPatch,
    "duration": 0.25,
    "active": isobar.PSequence([0, 0, 0, 0, 1, 0, 0, 0]),
}, quantize=1.0, name="clap", replace=True)

In [ ]:
timeline.clear()